In [1]:
# Melakukan Import Library Random, Serta Math untuk operasi sin dan cos
import random
from math import sin, cos

# Membuat array untuk data limit x dan y
x_limit = [-1, 2]
y_limit = [-1, 1]

In [2]:
# Fungsi pembuatan populasi beserta kromosomnya
def generate_population():
    population = []

    for _ in range(6):
        kromosom = []
        for _ in range(6) :
            kromosom.append(random.randint(0,9))
            
        population.append(kromosom)

    return population

In [3]:
# Fungsi membagi kromosom menjadi 2 bagian
def split_kromosom(kromosom):
    split = len(kromosom) // 2
    return kromosom[:split], kromosom[split:] 

In [4]:
# Fungsi rumus yang akan dicari nilai maksimumnya
def function(x,y):
    return (cos(x**2) * sin(y**2)) + (x + y)

In [5]:
# Fungsi decode untuk setiap kromosom pada populasi
def decode(kromosom, limit) :
    kali, pembagi = 0, 0
    for i in range(len(kromosom)) :
        num = kromosom[i]
        kali += num * (10**-(i+1))
        pembagi += 9 * (10**-(i+1))

    return limit[0] + (((limit[1] - limit[0]) / pembagi) * kali) 

In [6]:
# Fungsi untuk menentukan kromosom terbaik
def best_population(population):
    max_fitness = -999
    
    for kromosom in population:
        kromosom_a, kromosom_b = split_kromosom(kromosom)
        x1 = decode(kromosom_a, x_limit)
        x2 = decode(kromosom_b, y_limit)
        fitness = function(x1, x2)
        
        if  max_fitness < fitness:
            max_fitness = fitness
            max_kromosom = kromosom
      
    return max_kromosom, max_fitness, x1, x2

In [7]:
# Fungsi untuk melakukan proses seleksi orang tua
def parent_roulette_selection(population, fitness, fitness_total):
    r = random.random()
    i = 0
    while r > 0:
      r -= fitness[i]/fitness_total
      i += 1
      if  i == len(population) - 1:
          break
          
    return population[i]

In [8]:
# Fungsi untuk melakukan proses crossover anak
def crossover(parent_1, parent_2) :         
    child_1, child_2, childs = [], [], []
    pb = random.random()

    if pb < 0.9:
      child_1[:1], child_1[1:] = parent_1[:1], parent_2[1:]   
      child_2[:1], child_2[1:] = parent_2[:1], parent_1[1:]
      childs.append(child_1)
      childs.append(child_2)
    else:   
      childs.append(parent_1)
      childs.append(parent_2)

    return childs

In [9]:
# Fungsi untuk melakukan proses mutasi anak
def mutation(child_1, child_2):
    for i in range(len(child_1)):
        p = random.random()
        if p < 0.1:
           child_1[i] = random.randint(0,9)

        q = random.random()
        if q < 0.1:
           child_2[i] = random.randint(0,9)

    mutasi = []
    mutasi.append(child_1)
    mutasi.append(child_2)
    return mutasi

In [10]:
def elitisme(population, best_kromosom_generation, bad_kromosom, total_fitness):
    if best_kromosom_generation[1] > bad_kromosom[0]:
       if best_kromosom_generation[0] not in population:
          population[bad_kromosom[2]] = best_kromosom_generation[0]
          total_fitness = (total_fitness - bad_kromosom[0]) + best_kromosom_generation[1]
          print('-----------------------------------------------------------------------------------------') 
          print('Proses Elitisme')
          print(f'Kromosom Ke-{bad_kromosom[2]+1}: {bad_kromosom[1]}, fitness: {bad_kromosom[0]} dirubah menjadi {best_kromosom_generation[0]}, fitness: {best_kromosom_generation[1]}')
          print(f'Hasil Populasi: {population}')
          print('-----------------------------------------------------------------------------------------')

    return population, total_fitness

In [11]:
# Inisialisasi Jumlah populasi, generasi serta pemanggilan fungsi untuk membuat populasi
population_total = 6
generation = 6
population = generate_population()
print("Populasi Awal:", population, "\n")

best_kromosom_generation = []

# Perulangan untuk melakukan proses seleksi populasi
for gen in range(generation):

      # Inisialisasi variabel untuk proses perhitungan algoritma genetika
      kromosom, best_kromosom, bad_kromosom, fitness_data, new_population, child = [], [], [], [], [], []
      total_fitness, count_kromosom, index = 0, 999, 0

      print('-----------------------------------------------------------------------------------------')
      print('Populasi Generasi', gen+1)
      print('-----------------------------------------------------------------------------------------')

      # Perulanagan untuk mencari nilai phenotype dan nilai fungsi / fitness pada setiap kromosom 
      for i, kromosom in enumerate(population):                         
          kromosom_a, kromosom_b = split_kromosom(kromosom)
          x1 = decode(kromosom_a, x_limit)
          x2 = decode(kromosom_b, y_limit)

          fitness_value = function(x1, x2)
          fitness_data.append(fitness_value)
          total_fitness += fitness_value
          
          if gen != 0:
             if fitness_value < count_kromosom:
                count_kromosom = fitness_value
                bad_kromosom = [fitness_value, kromosom, i]
          
          print(f'Kromosom {i+1}              = {kromosom}')
          print('Nilai Phenotype X1      =', x1)
          print('Nilai Phenotype X2      =', x2)
          print('Nilai Fungsi / Fitness  =', fitness_value, '\n')
      
      if gen != 0:
         population, total_fitness = elitisme(population, best_kromosom_generation[-1], bad_kromosom, total_fitness)

      print('Total Nilai Fitness =', total_fitness)
      print('-----------------------------------------------------------------------------------------')
      print('\nSeleksi Orang Tua Dengan Roulette Wheel\n')

      # Perulangan untuk melakukan seleksi orang tua, crossover dan mutasi anak untuk mendapatkan populasi generasi selanjutnya
      for i in range(population_total // 2):
          parent_1 = parent_roulette_selection(population, fitness_data, total_fitness)
          parent_2 = parent_roulette_selection(population, fitness_data, total_fitness)
          print(f'Iterasi ke-{i+1}:')
          print('Hasil Seleksi Parent 1 = ', parent_1)
          print('Hasil Seleksi Parent 2 = ', parent_2)

          child = crossover(parent_1, parent_2)
          print('Hasil Crossover Child  = ', child, '\n')

          child = mutation(child[0], child[1])
          new_population.append(child[0])
          new_population.append(child[1])

      print('Populasi Baru Generasi', gen+2, ' = ', new_population)
      print('------------------------------------------------------------------------------------\n')
      
      best_kromosom_generation.append(best_population(population))
      population = new_population
      generation += 1

print('-----------------------------------------------------------------------------------------')
print('Populasi Akhir Generasi', gen+2)
print('-----------------------------------------------------------------------------------------')

# Perulangan untuk mencari nilai phenotype dan nilai fungsi / fitness pada setiap kromosom generasi terakhir
for i, kromosom in enumerate(population):                         
          kromosom_a, kromosom_b = split_kromosom(kromosom)
          x1 = decode(kromosom_a, x_limit)
          x2 = decode(kromosom_b, y_limit)

          fitness_value = function(x1, x2)
          fitness_data.append(fitness_value)
          total_fitness += fitness_value
          
          print(f'Kromosom {i+1}              = {kromosom}')
          print('Nilai Phenotype X1      =', x1)
          print('Nilai Phenotype X2      =', x2)
          print('Nilai Fungsi / Fitness  =', fitness_value, '\n')

# Memanggil fungsi untuk menentukan kromosom terbaik pada generasi terakhir
population, total_fitness = elitisme(population, best_kromosom_generation[-1], bad_kromosom, total_fitness)
best_kromosom, fitness_value, x1, x2 = best_population(population)

print('\n-----------------------------------------------------')
print('Hasil Akhir Kromosom Terbaik')
print('-----------------------------------------------------')
print('Kromosom Terbaik         = ', best_kromosom)
print('Phenotype x1             = ', x1)
print('Phenotype x2             = ', x2)
print('Nilai Fungsi / Fitness   = ', fitness_value)
print('-----------------------------------------------------')

Populasi Awal: [[7, 7, 1, 4, 9, 4], [6, 4, 9, 9, 9, 6], [1, 2, 1, 9, 3, 3], [4, 2, 3, 9, 7, 9], [0, 5, 2, 2, 3, 9], [8, 6, 2, 1, 1, 9]] 

-----------------------------------------------------------------------------------------
Populasi Generasi 1
-----------------------------------------------------------------------------------------
Kromosom 1              = [7, 7, 1, 4, 9, 4]
Nilai Phenotype X1      = 1.3153153153153152
Nilai Phenotype X2      = -0.011011011011010985
Nilai Fungsi / Fitness  = 1.3042850770003074 

Kromosom 2              = [6, 4, 9, 9, 9, 6]
Nilai Phenotype X1      = 0.9489489489489493
Nilai Phenotype X2      = 0.9939939939939941
Nilai Fungsi / Fitness  = 2.461620306880839 

Kromosom 3              = [1, 2, 1, 9, 3, 3]
Nilai Phenotype X1      = -0.6366366366366366
Nilai Phenotype X2      = 0.8678678678678682
Nilai Fungsi / Fitness  = 0.859789680457665 

Kromosom 4              = [4, 2, 3, 9, 7, 9]
Nilai Phenotype X1      = 0.2702702702702704
Nilai Phenotype X2      